<a href="https://colab.research.google.com/github/NearANDfar13/NLP_Winter_2024/blob/main/Final_Notebooks_San_Fran/NAA_San_Fran_NER_ImpressoAPI_Pub_Place.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NearANDfar13/NLP_Winter_2024.git

Cloning into 'NLP_Winter_2024'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 222 (delta 31), reused 2 (delta 2), pack-reused 150 (from 1)
Receiving objects: 100% (222/222), 13.19 MiB | 12.88 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [2]:
#From NLP class on Impresso API
#Data organizing and filtering method from NLP class
#Analysis of Publication Locations of historical German newspapers
#Import the dataset "Extended Sample"
import pandas as pd

articles_df = pd.read_excel('/content/NLP_Winter_2024/NA_San_Fran_Extended_Sample.xlsx')

#Display DataFrame

articles_df.head()

,ID,Day,Short_Name,Full_Name,Place,Frequency,Publication,Text,Page,Link,Datengeber
0,3043262-5,19/04/1906,Annener Zeitung,"Annener Zeitung, verbunden mit der Annener Vol...","Annen (Witten), Witten-Annen",NaN,1885-1943,"New=York, 14 Francisco von eine gesucht. Tause...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,Westfälische Wilhelms-Universität Münster Univ...
1,3118927-1,19/04/1906,Anzeiger vom Oberland,"Anzeiger vom Oberland, Tageszeitung für das Ob...",Biberach an der Riß,täglich,1870-1932,"au- Oakland als Gerücht gemeldet, daß da- amer...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,Kreisarchiv des Landkreises Biberach
2,3118927-1,19/04/1906,Anzeiger vom Oberland,"Anzeiger vom Oberland, Tageszeitung für das Ob...",Biberach an der Riß,täglich,1870-1932,"ertappt wird, zu erschießen. ir Gf Store (CaUf...",6.,https://www.deutsche-digitale-bibliothek.de/ne...,Kreisarchiv des Landkreises Biberach
3,2797055-3,19/04/1906,Badische Presse,Badische Presse : Generalanzeiger der Residenz...,Karlsruhe,täglich,1890-1944,Die Marinestationen am Süllen Ozean versuchten...,4.,https://www.deutsche-digitale-bibliothek.de/ne...,Badische Landesbibliothek
4,2958285-4,19/04/1906,Der Beobachter,"Der Beobachter, ein Volksblatt aus Schwaben","Stuttgart, Württemberg",täglich,1833-1920,"Erdbeben in Ealifornien. New-Sjork, 18. April....",3.,https://www.deutsche-digitale-bibliothek.de/ne...,Württembergische Landesbibliothek


In [3]:
#Visualizaing Edited List of NER Locations as a Map
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import pandas as pd
import time
from typing import List, Tuple, Optional
import random

class GeocodingService:
    def __init__(self, user_agent: str = None, timeout: int = 10, rate_limit: float = 1.1):
        """
        Initialize the geocoding service with proper configuration.

        Args:
            user_agent: Custom user agent string (default: generated)
            timeout: Timeout for requests in seconds
            rate_limit: Time to wait between requests in seconds
        """
        if user_agent is None:
            user_agent = f"python_geocoding_script_{random.randint(1000, 9999)}"

        self.geolocator = Nominatim(
            user_agent=user_agent,
            timeout=timeout
        )
        self.rate_limit = rate_limit
        self.last_request = 0

    def _rate_limit_wait(self):
        """Implement rate limiting between requests"""
        current_time = time.time()
        time_since_last = current_time - self.last_request
        if time_since_last < self.rate_limit:
            time.sleep(self.rate_limit - time_since_last)
        self.last_request = time.time()

    def geocode_location(self, location: str, max_retries: int = 3) -> Optional[Tuple[float, float]]:
        """
        Geocode a single location with retries.

        Args:
            location: Location string to geocode
            max_retries: Maximum number of retry attempts

        Returns:
            Tuple of (latitude, longitude) or None if geocoding fails
        """
        for attempt in range(max_retries):
            try:
                self._rate_limit_wait()
                location_data = self.geolocator.geocode(location)
                if location_data:
                    return (location_data.latitude, location_data.longitude)
                return None
            except (GeocoderTimedOut, GeocoderServiceError) as e:
                if attempt == max_retries - 1:
                    print(f"Failed to geocode '{location}' after {max_retries} attempts: {e}")
                    return None
                time.sleep(2 ** attempt)  # Exponential backoff
            except Exception as e:
                print(f"Error geocoding '{location}': {e}")
                return None
        return None

    def process_locations(self, locations: str) -> List[Optional[Tuple[float, float]]]:
        """
        Process a comma-separated string of locations.

        Args:
            locations: Comma-separated string of location names

        Returns:
            List of coordinate tuples or None for failed geocoding
        """
        if pd.isna(locations) or not locations:
            return []

        location_list = [loc.strip() for loc in locations.split(',')]
        return [self.geocode_location(loc) for loc in location_list]

def geolocate_places(df: pd.DataFrame,
                    places_column: str = 'Place',
                    user_agent: str = None) -> pd.DataFrame:
    """
    Add coordinates to a DataFrame based on location names.

    Args:
        df: Input DataFrame
        places_column: Name of the column containing comma-separated location strings
        user_agent: Custom user agent string

    Returns:
        DataFrame with added 'coordinates' column
    """
    geocoder = GeocodingService(user_agent=user_agent)

    # Create a copy to avoid modifying the original DataFrame
    result_df = df.copy()

    # Process locations
    result_df['pub_coordinates'] = result_df[places_column].apply(geocoder.process_locations)

    return result_df

# Main execution
if __name__ == "__main__":
    # Assuming articles_df is your DataFrame with a 'places' column
    # Apply geocoding to the articles DataFrame
    articles_df_with_coords = geolocate_places(
        articles_df,
        places_column='Place',
        user_agent='article_geocoding_service_v1.0'
    )

    # Update the original DataFrame with the new coordinates
    articles_df['pub_coordinates'] = articles_df_with_coords['pub_coordinates']

    # Display the results
    print("\nSample of geocoded locations:")
    print(articles_df[['Place', 'pub_coordinates']].head())

    # Optional: Display some statistics
    total_locations = len(articles_df)
    successful_geocodes = articles_df['pub_coordinates'].apply(lambda x: len([c for c in x if c is not None])).sum()
    failed_geocodes = articles_df['pub_coordinates'].apply(lambda x: len([c for c in x if c is None])).sum()

    print(f"\nGeocoding Statistics:")
    print(f"Total locations processed: {total_locations}")
    print(f"Successfully geocoded: {successful_geocodes}")
    print(f"Failed to geocode: {failed_geocodes}")


Sample of geocoded locations:
                          Place  \
0  Annen (Witten), Witten-Annen   
1           Biberach an der Riß   
2           Biberach an der Riß   
3                     Karlsruhe   
4        Stuttgart, Württemberg   

                                     pub_coordinates  
0  [(51.4462436, 7.3744351), (51.4462436, 7.37443...  
1                          [(48.0984413, 9.7899938)]  
2                          [(48.0984413, 9.7899938)]  
3                          [(49.0068705, 8.4034195)]  
4  [(48.7784485, 9.1800132), (48.3366504, 9.09940...  

Geocoding Statistics:
Total locations processed: 310
Successfully geocoded: 591
Failed to geocode: 18


In [5]:
# export as excel file
articles_df.to_excel('earthquake_articles_pub_places.xlsx', index=False)

In [4]:
import folium
from folium import plugins
import pandas as pd
from typing import List, Tuple, Optional
from IPython.display import display

def create_location_map(df: pd.DataFrame,
                       coordinates_col: str = 'pub_coordinates',
                       places_col: str = 'Place',
                       title_col: Optional[str] = None) -> folium.Map:
    """
    Create an interactive map with markers for all locations in the DataFrame.

    Args:
        df: DataFrame containing coordinates and place names
        coordinates_col: Name of column containing coordinates
        places_col: Name of column containing place names
        title_col: Optional column name for additional marker information

    Returns:
        folium.Map object with all locations marked
    """
    # Initialize the map
    m = folium.Map(location=[0, 0], zoom_start=2)

    # Create a MarkerCluster
    marker_cluster = plugins.MarkerCluster().add_to(m)

    # Keep track of all valid coordinates for setting bounds
    all_coords = []

    # Process each row in the DataFrame
    for idx, row in df.iterrows():
        coordinates = row[coordinates_col]
        places = row[places_col].split(',') if pd.notna(row[places_col]) else []
        title = row[title_col] if title_col and pd.notna(row[title_col]) else None

        # Skip if no coordinates
        if not coordinates:
            continue

        # Add markers for each location
        for i, (coord, place) in enumerate(zip(coordinates, places)):
            if coord is not None:  # Skip None coordinates
                lat, lon = coord
                place_name = place.strip()

                # Create popup content
                popup_content = f"<b>{place_name}</b>"
                if title:
                    popup_content += f"<br>{title}"

                # Add marker
                folium.Marker(
                    location=[lat, lon],
                    popup=popup_content,
                    tooltip=place_name
                ).add_to(marker_cluster)

                all_coords.append([lat, lon])

    # If we have coordinates, fit the map bounds to include all points
    if all_coords:
        m.fit_bounds(all_coords)

    return m

# Create and display the map
map_obj = create_location_map(articles_df)
display(map_obj)